In [7]:
import pandas as pd
import json
from sqlalchemy import create_engine
import psycopg2

pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', 100)

import numpy as np
import os

In [8]:
# Function to extract qualifiers into separate columns
def extract_qualifiers(df):
    # Initialize a list to hold all unique displayNames
    unique_display_names = set()

    # Iterate over each row in the qualifiers column to extract unique displayNames
    for item in df['qualifiers']:
        if isinstance(item, str):
            qualifiers_list = json.loads(item.replace("'", "\""))
            for qualifier in qualifiers_list:
                unique_display_names.add(qualifier['type']['displayName'])

    # Create a dictionary to hold the new columns data
    new_columns_data = {display_name: [] for display_name in unique_display_names}

    # Populate the new columns with corresponding values
    for item in df['qualifiers']:
        if isinstance(item, str):
            qualifiers_list = json.loads(item.replace("'", "\""))
            row_data = {display_name: None for display_name in unique_display_names}
            for qualifier in qualifiers_list:
                display_name = qualifier['type']['displayName']
                value = qualifier.get('value', None)
                row_data[display_name] = value
            for display_name in unique_display_names:
                new_columns_data[display_name].append(row_data[display_name])
        else:
            for display_name in unique_display_names:
                new_columns_data[display_name].append(None)

    # Create a new DataFrame from the new columns data
    new_columns_df = pd.DataFrame(new_columns_data)

    # Concatenate the new columns DataFrame with the original DataFrame
    df = pd.concat([df, new_columns_df], axis=1)

    return df

# MU 2022 - 2023 Events

In [3]:
#Read All events data
matchId = 1
player_list = pd.DataFrame()
events_all = pd.DataFrame()
for i in range(1,63):
    #Read json based on declared matcho
    read_json = open(f'whoscored_data/mu_data/2022_2023/{i}.json', encoding="utf-8")

    #Load json data
    data = json.load(read_json)

    #Put json data on dataframe and normalize it
    events = pd.json_normalize(data['events'])

    #Add matchId data to differentiate between matches
    events['match_id']=matchId
    matchId = matchId + 1

    #append all events data into 1 dataframe
    events_all = pd.concat([events_all,events],ignore_index=True)

    #read player name and put into list
    player_name = pd.json_normalize(data['playerIdNameDictionary']).transpose()
    player_name = player_name.reset_index()
    player_name = player_name.rename(columns={"index":"playerId",
                                                0:"player_name",
                                            })
    player_name['playerId'] = player_name['playerId'].astype(int, errors = 'raise')
    player_list = pd.concat([player_list,player_name]).drop_duplicates().reset_index(drop=True)

#get player name into events data
events_all = events_all.merge(player_list,on=['playerId'],how='left')
events_all = events_all.rename(columns= {"eventId":"event_id",
                                        "teamId":"team_id",
                                        "playerId":"player_id",
                                        "expandedMinute":"expanded_minute",
                                        "satisfiedEventsTypes":"satisfied_events_types",
                                        "isTouch":"is_touch",
                                        "period.value":"period_value",
                                        "period.displayName":"period_display_name",
                                        "type.value":"type_value",
                                        "type.displayName":"type_display_name",
                                        "outcomeType.value":"outcome_type_value",
                                        "outcomeType.displayName":"outcome_type_display_name",
                                        "endX":"end_x",
                                        "endY":"end_y",
                                        "goalMouthZ":"goal_mouth_z",
                                        "goalMouthY":"goal_mouth_y",
                                        "isShot":"is_shot",
                                        "relatedEventId":"related_event_id",
                                        "relatedPlayerId":"related_player_id",
                                        "blockedX":"blocked_x",
                                        "blockedY":"blocked_y",
                                        "isGoal":"is_goal",
                                        "isOwnGoal":"is_own_goal",
                                        "cardType.value":"card_type_value",
                                        "cardtype_display_name":"card_type_display_name"
                                        })
events_all['qualifiers'] = events_all['qualifiers'].apply(json.dumps)
events_all = extract_qualifiers(events_all)
events_all = events_all.drop('qualifiers',axis=1)
events_all

,id,event_id,minute,second,team_id,x,y,expanded_minute,satisfied_events_types,is_touch,period_value,period_display_name,type_value,type_display_name,outcome_type_value,outcome_type_display_name,player_id,end_x,end_y,goal_mouth_z,goal_mouth_y,is_shot,related_event_id,related_player_id,blocked_x,blocked_y,card_type_value,cardType.displayName,is_goal,is_own_goal,match_id,player_name,BlockedCross,HeadPass,ParriedSafe,PenaltyShootoutConcededGK,OppositeRelatedEvent,PassEndX,PassEndY,DivingSave,GoalMouthY,Cross,HighLeft,SmallBoxLeft,OutOfBoxRight,OutOfBoxLeft,OutOfBoxCentre,ThrowinSetPiece,KeeperSaveObox,PlayerCaughtOffside,FormationSlot,Longball,GoalMouthZ,IntentionalGoalAssist,Red,BigChanceCreated,SetPiece,BlockedX,Length,VoidYellowCard,Obstruction,BoxLeft,ParriedDanger,StandingSave,PlayerPosition,DeepBoxRight,Foul,MissLeft,Head,ShotAssist,SixYardBlock,LeftFoot,Assisted,MissHigh,BoxRight,RegularPlay,JerseyNumber,Yellow,ThirtyFivePlusRight,MissRight,GoalDisallowed,KeeperSaveInSixYard,Offensive,Zone,BoxCentre,FromShotOffTarget,Throughball,OutfielderBlock,CaptainPlayerId,Blocked,TeamPlayerFormation,RightFoot,Hands,FromCorner,OutOfBoxDeepRight,Angle,FastBreak,OtherBodyPart,IndirectFreekickTaken,Feet,OutOfBoxDeepLeft,LeadingToAttempt,GoalKick,BigChance,InvolvedPlayers,LastMan,BlockedY,Defensive,HighCentre,Penalty,Collected,LowRight,DeepBoxLeft,Chipped,LowCentre,SmallBoxCentre,KeeperSaved,KeyPass,LowLeft,SmallBoxRight,RelatedEventId,ThrowIn,SavedOffline,IntentionalAssist,OwnGoal,CornerTaken,OverRun,LayOff,ThirtyFivePlusCentre,KeeperWentWide,KeeperThrow,HighClaim,HighRight,KeeperMissed,KeeperSaveInTheBox,TeamFormation,DirectFreekick,AerialFoul,SecondYellow,FreekickTaken,LeadingToGoal
0,2.443300e+09,2,0,0.0,211,0.0,0.0,0,[],False,1,FirstHalf,32,Start,1,Successful,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
1,2.443300e+09,2,0,0.0,32,0.0,0.0,0,[],False,1,FirstHalf,32,Start,1,Successful,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
2,2.443300e+09,3,0,0.0,211,50.0,50.0,0,"[91, 117, 30, 35, 38, 215, 218]",True,1,FirstHalf,1,Pass,1,Successful,21683.0,30.7,46.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,Adam Lallana,None,None,None,None,None,30.7,46.9,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,20.4,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,Back,None,None,None,None,None,None,None,None,None,None,None,3.2,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
3,2.443300e+09,4,0,3.0,211,31.6,43.4,0,"[91, 120, 124, 128, 36, 38, 217, 218]",True,1,FirstHalf,1,Pass,0,Unsuccessful,86441.0,78.3,26.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,Lewis Dunk,No

In [4]:
events_mu = events_all.loc[events_all.team_id==32].reset_index(drop=True)
# events_mu['pass_recipient_name'] = np.nan
# events_mu['pass_recipient_id'] = np.nan
events_mu

,id,event_id,minute,second,team_id,x,y,expanded_minute,satisfied_events_types,is_touch,period_value,period_display_name,type_value,type_display_name,outcome_type_value,outcome_type_display_name,player_id,end_x,end_y,goal_mouth_z,goal_mouth_y,is_shot,related_event_id,related_player_id,blocked_x,blocked_y,card_type_value,cardType.displayName,is_goal,is_own_goal,match_id,player_name,BlockedCross,HeadPass,ParriedSafe,PenaltyShootoutConcededGK,OppositeRelatedEvent,PassEndX,PassEndY,DivingSave,GoalMouthY,Cross,HighLeft,SmallBoxLeft,OutOfBoxRight,OutOfBoxLeft,OutOfBoxCentre,ThrowinSetPiece,KeeperSaveObox,PlayerCaughtOffside,FormationSlot,Longball,GoalMouthZ,IntentionalGoalAssist,Red,BigChanceCreated,SetPiece,BlockedX,Length,VoidYellowCard,Obstruction,BoxLeft,ParriedDanger,StandingSave,PlayerPosition,DeepBoxRight,Foul,MissLeft,Head,ShotAssist,SixYardBlock,LeftFoot,Assisted,MissHigh,BoxRight,RegularPlay,JerseyNumber,Yellow,ThirtyFivePlusRight,MissRight,GoalDisallowed,KeeperSaveInSixYard,Offensive,Zone,BoxCentre,FromShotOffTarget,Throughball,OutfielderBlock,CaptainPlayerId,Blocked,TeamPlayerFormation,RightFoot,Hands,FromCorner,OutOfBoxDeepRight,Angle,FastBreak,OtherBodyPart,IndirectFreekickTaken,Feet,OutOfBoxDeepLeft,LeadingToAttempt,GoalKick,BigChance,InvolvedPlayers,LastMan,BlockedY,Defensive,HighCentre,Penalty,Collected,LowRight,DeepBoxLeft,Chipped,LowCentre,SmallBoxCentre,KeeperSaved,KeyPass,LowLeft,SmallBoxRight,RelatedEventId,ThrowIn,SavedOffline,IntentionalAssist,OwnGoal,CornerTaken,OverRun,LayOff,ThirtyFivePlusCentre,KeeperWentWide,KeeperThrow,HighClaim,HighRight,KeeperMissed,KeeperSaveInTheBox,TeamFormation,DirectFreekick,AerialFoul,SecondYellow,FreekickTaken,LeadingToGoal
0,2.443300e+09,2,0,0.0,32,0.0,0.0,0,[],False,1,FirstHalf,32,Start,1,Successful,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
1,2.443300e+09,3,0,6.0,32,20.8,78.7,0,[93],False,1,FirstHalf,49,BallRecovery,1,Successful,340105.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,Lisandro Martínez,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
2,2.443300e+09,4,0,8.0,32,18.4,63.9,0,"[91, 117, 30, 35, 38, 215, 218]",True,1,FirstHalf,1,Pass,1,Successful,340105.0,18.3,37.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,Lisandro Martínez,None,None,None,None,None,18.3,37.6,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,17.9,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,Back,None,None,None,None,None,None,None,None,None,None,None,4.7,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
3,2.443300e+09,5,0,10.0,32,17.7,30.4,0,"[91, 118, 120, 29, 36, 38, 215, 218]",True,1,FirstHalf,1,Pass,0,Unsuccessful,353418.0,19.2,25.5,NaN,NaN,NaN,NaN,NaN,NaN,N

In [5]:
pass_success = events_mu.loc[(events_mu.type_display_name=='Pass') & (events_mu.outcome_type_display_name=='Successful')]
for i in pass_success.index:
    events_mu.loc[i, ['pass_recipient_name']]= events_mu.iloc[i+1]['player_name']
    events_mu.loc[i, ['pass_recipient_id']] = events_mu.iloc[i+1]['player_id']
events_mu

,id,event_id,minute,second,team_id,x,y,expanded_minute,satisfied_events_types,is_touch,period_value,period_display_name,type_value,type_display_name,outcome_type_value,outcome_type_display_name,player_id,end_x,end_y,goal_mouth_z,goal_mouth_y,is_shot,related_event_id,related_player_id,blocked_x,blocked_y,card_type_value,cardType.displayName,is_goal,is_own_goal,match_id,player_name,BlockedCross,HeadPass,ParriedSafe,PenaltyShootoutConcededGK,OppositeRelatedEvent,PassEndX,PassEndY,DivingSave,GoalMouthY,Cross,HighLeft,SmallBoxLeft,OutOfBoxRight,OutOfBoxLeft,OutOfBoxCentre,ThrowinSetPiece,KeeperSaveObox,PlayerCaughtOffside,FormationSlot,Longball,GoalMouthZ,IntentionalGoalAssist,Red,BigChanceCreated,SetPiece,BlockedX,Length,VoidYellowCard,Obstruction,BoxLeft,ParriedDanger,StandingSave,PlayerPosition,DeepBoxRight,Foul,MissLeft,Head,ShotAssist,SixYardBlock,LeftFoot,Assisted,MissHigh,BoxRight,RegularPlay,JerseyNumber,Yellow,ThirtyFivePlusRight,MissRight,GoalDisallowed,KeeperSaveInSixYard,Offensive,Zone,BoxCentre,FromShotOffTarget,Throughball,OutfielderBlock,CaptainPlayerId,Blocked,TeamPlayerFormation,RightFoot,Hands,FromCorner,OutOfBoxDeepRight,Angle,FastBreak,OtherBodyPart,IndirectFreekickTaken,Feet,OutOfBoxDeepLeft,LeadingToAttempt,GoalKick,BigChance,InvolvedPlayers,LastMan,BlockedY,Defensive,HighCentre,Penalty,Collected,LowRight,DeepBoxLeft,Chipped,LowCentre,SmallBoxCentre,KeeperSaved,KeyPass,LowLeft,SmallBoxRight,RelatedEventId,ThrowIn,SavedOffline,IntentionalAssist,OwnGoal,CornerTaken,OverRun,LayOff,ThirtyFivePlusCentre,KeeperWentWide,KeeperThrow,HighClaim,HighRight,KeeperMissed,KeeperSaveInTheBox,TeamFormation,DirectFreekick,AerialFoul,SecondYellow,FreekickTaken,LeadingToGoal,pass_recipient_name,pass_recipient_id
0,2.443300e+09,2,0,0.0,32,0.0,0.0,0,[],False,1,FirstHalf,32,Start,1,Successful,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,NaN,NaN
1,2.443300e+09,3,0,6.0,32,20.8,78.7,0,[93],False,1,FirstHalf,49,BallRecovery,1,Successful,340105.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,Lisandro Martínez,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,NaN,NaN
2,2.443300e+09,4,0,8.0,32,18.4,63.9,0,"[91, 117, 30, 35, 38, 215, 218]",True,1,FirstHalf,1,Pass,1,Successful,340105.0,18.3,37.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,Lisandro Martínez,None,None,None,None,None,18.3,37.6,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,17.9,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,Back,None,None,None,None,None,None,None,None,None,None,None,4.7,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,Diogo Dalot,353418.0
3,2.443300e+09,5,0,10.0,32,17.7,30.4,0,"[91, 118, 120, 29, 36, 38, 215, 218]",True,1,F

In [6]:
events_mu.to_csv('Manchester United Events 2022-2023 Preprocessed.csv')

# MU 2023 - 2024 Events

In [ ]:
#Read All events data
matchId = 1
player_list = pd.DataFrame()
events_all = pd.DataFrame()
for i in range(1,53):
    #Read json based on declared matcho
    read_json = open(f'whoscored_data/mu_data/2023_2024/{i}.json', encoding="utf-8")

    #Load json data
    data = json.load(read_json)

    #Put json data on dataframe and normalize it
    events = pd.json_normalize(data['events'])

    #Add matchId data to differentiate between matches
    events['match_id']=matchId
    matchId = matchId + 1

    #append all events data into 1 dataframe
    events_all = pd.concat([events_all,events],ignore_index=True)

    #read player name and put into list
    player_name = pd.json_normalize(data['playerIdNameDictionary']).transpose()
    player_name = player_name.reset_index()
    player_name = player_name.rename(columns={"index":"playerId",
                                                0:"player_name",
                                            })
    player_name['playerId'] = player_name['playerId'].astype(int, errors = 'raise')
    player_list = pd.concat([player_list,player_name]).drop_duplicates().reset_index(drop=True)

#get player name into events data
events_all = events_all.merge(player_list,on=['playerId'],how='left')
events_all = events_all.rename(columns= {"eventId":"event_id",
                                        "teamId":"team_id",
                                        "playerId":"player_id",
                                        "expandedMinute":"expanded_minute",
                                        "satisfiedEventsTypes":"satisfied_events_types",
                                        "isTouch":"is_touch",
                                        "period.value":"period_value",
                                        "period.displayName":"period_display_name",
                                        "type.value":"type_value",
                                        "type.displayName":"type_display_name",
                                        "outcomeType.value":"outcome_type_value",
                                        "outcomeType.displayName":"outcome_type_display_name",
                                        "endX":"end_x",
                                        "endY":"end_y",
                                        "goalMouthZ":"goal_mouth_z",
                                        "goalMouthY":"goal_mouth_y",
                                        "isShot":"is_shot",
                                        "relatedEventId":"related_event_id",
                                        "relatedPlayerId":"related_player_id",
                                        "blockedX":"blocked_x",
                                        "blockedY":"blocked_y",
                                        "isGoal":"is_goal",
                                        "isOwnGoal":"is_own_goal",
                                        "cardType.value":"card_type_value",
                                        "cardtype_display_name":"card_type_display_name"
                                        })
events_all['qualifiers'] = events_all['qualifiers'].apply(json.dumps)
events_all = extract_qualifiers(events_all)
events_all = events_all.drop('qualifiers',axis=1)
events_all

In [ ]:
events_mu = events_all.loc[events_all.team_id==32].reset_index(drop=True)
# events_mu['pass_recipient_name'] = np.nan
# events_mu['pass_recipient_id'] = np.nan
events_mu

In [ ]:
pass_success = events_mu.loc[(events_mu.type_display_name=='Pass') & (events_mu.outcome_type_display_name=='Successful')]
for i in pass_success.index:
    events_mu.loc[i, ['pass_recipient_name']]= events_mu.iloc[i+1]['player_name']
    events_mu.loc[i, ['pass_recipient_id']] = events_mu.iloc[i+1]['player_id']
events_mu

In [ ]:
events_mu.to_csv('Manchester United Events 2023-2024 Preprocessed.csv')

# Bologna Events

In [ ]:
#Read All events data
matchId = 1
player_list = pd.DataFrame()
events_all = pd.DataFrame()
for i in range(1,35):
    #Read json based on declared matcho
    read_json = open(f'whoscored_data/zirkzee_data/{i}.json', encoding="utf-8")

    #Load json data
    data = json.load(read_json)

    #Put json data on dataframe and normalize it
    events = pd.json_normalize(data['events'])

    #Add matchId data to differentiate between matches
    events['match_id']=matchId
    matchId = matchId + 1

    #append all events data into 1 dataframe
    events_all = pd.concat([events_all,events],ignore_index=True)

    #read player name and put into list
    player_name = pd.json_normalize(data['playerIdNameDictionary']).transpose()
    player_name = player_name.reset_index()
    player_name = player_name.rename(columns={"index":"playerId",
                                                0:"player_name",
                                            })
    player_name['playerId'] = player_name['playerId'].astype(int, errors = 'raise')
    player_list = pd.concat([player_list,player_name]).drop_duplicates().reset_index(drop=True)

#get player name into events data
events_all = events_all.merge(player_list,on=['playerId'],how='left')
events_all = events_all.rename(columns= {"eventId":"event_id",
                                        "teamId":"team_id",
                                        "playerId":"player_id",
                                        "expandedMinute":"expanded_minute",
                                        "satisfiedEventsTypes":"satisfied_events_types",
                                        "isTouch":"is_touch",
                                        "period.value":"period_value",
                                        "period.displayName":"period_display_name",
                                        "type.value":"type_value",
                                        "type.displayName":"type_display_name",
                                        "outcomeType.value":"outcome_type_value",
                                        "outcomeType.displayName":"outcome_type_display_name",
                                        "endX":"end_x",
                                        "endY":"end_y",
                                        "goalMouthZ":"goal_mouth_z",
                                        "goalMouthY":"goal_mouth_y",
                                        "isShot":"is_shot",
                                        "relatedEventId":"related_event_id",
                                        "relatedPlayerId":"related_player_id",
                                        "blockedX":"blocked_x",
                                        "blockedY":"blocked_y",
                                        "isGoal":"is_goal",
                                        "isOwnGoal":"is_own_goal",
                                        "cardType.value":"card_type_value",
                                        "cardtype_display_name":"card_type_display_name"
                                        })
events_all['qualifiers'] = events_all['qualifiers'].apply(json.dumps)
events_all = extract_qualifiers(events_all)
events_all = events_all.drop('qualifiers',axis=1)
events_all

In [ ]:
bologna_events = events_all.loc[events_all.team_id==71].reset_index(drop=True)
bologna_events

In [ ]:
pass_success = bologna_events.loc[(bologna_events.type_display_name=='Pass') & (bologna_events.outcome_type_display_name=='Successful')]
for i in pass_success.index:
    bologna_events.loc[i, ['pass_recipient_name']]= bologna_events.iloc[i+1]['player_name']
    bologna_events.loc[i, ['pass_recipient_id']] = bologna_events.iloc[i+1]['player_id']
bologna_events

In [ ]:
bologna_events.to_csv('Bologna Events 2023-2024 Preprocessed.csv')

# Lille Events

In [9]:
#Read All events data
matchId = 1
player_list = pd.DataFrame()
events_all = pd.DataFrame()
for i in range(1,33):
    #Read json based on declared matcho
    read_json = open(f'whoscored_data/lille_data/{i}.json', encoding="utf-8")

    #Load json data
    data = json.load(read_json)

    #Put json data on dataframe and normalize it
    events = pd.json_normalize(data['events'])

    #Add matchId data to differentiate between matches
    events['match_id']=matchId
    matchId = matchId + 1

    #append all events data into 1 dataframe
    events_all = pd.concat([events_all,events],ignore_index=True)

    #read player name and put into list
    player_name = pd.json_normalize(data['playerIdNameDictionary']).transpose()
    player_name = player_name.reset_index()
    player_name = player_name.rename(columns={"index":"playerId",
                                                0:"player_name",
                                            })
    player_name['playerId'] = player_name['playerId'].astype(int, errors = 'raise')
    player_list = pd.concat([player_list,player_name]).drop_duplicates().reset_index(drop=True)

#get player name into events data
events_all = events_all.merge(player_list,on=['playerId'],how='left')
events_all = events_all.rename(columns= {"eventId":"event_id",
                                        "teamId":"team_id",
                                        "playerId":"player_id",
                                        "expandedMinute":"expanded_minute",
                                        "satisfiedEventsTypes":"satisfied_events_types",
                                        "isTouch":"is_touch",
                                        "period.value":"period_value",
                                        "period.displayName":"period_display_name",
                                        "type.value":"type_value",
                                        "type.displayName":"type_display_name",
                                        "outcomeType.value":"outcome_type_value",
                                        "outcomeType.displayName":"outcome_type_display_name",
                                        "endX":"end_x",
                                        "endY":"end_y",
                                        "goalMouthZ":"goal_mouth_z",
                                        "goalMouthY":"goal_mouth_y",
                                        "isShot":"is_shot",
                                        "relatedEventId":"related_event_id",
                                        "relatedPlayerId":"related_player_id",
                                        "blockedX":"blocked_x",
                                        "blockedY":"blocked_y",
                                        "isGoal":"is_goal",
                                        "isOwnGoal":"is_own_goal",
                                        "cardType.value":"card_type_value",
                                        "cardtype_display_name":"card_type_display_name"
                                        })
events_all['qualifiers'] = events_all['qualifiers'].apply(json.dumps)
events_all = extract_qualifiers(events_all)
events_all = events_all.drop('qualifiers',axis=1)
events_all

,id,event_id,minute,second,team_id,x,y,expanded_minute,satisfied_events_types,is_touch,period_value,period_display_name,type_value,type_display_name,outcome_type_value,outcome_type_display_name,player_id,end_x,end_y,blocked_x,blocked_y,goal_mouth_z,goal_mouth_y,is_shot,related_event_id,related_player_id,card_type_value,cardType.displayName,is_goal,match_id,is_own_goal,player_name,BlockedCross,HeadPass,ParriedSafe,OppositeRelatedEvent,IndividualPlay,PassEndX,PassEndY,DivingSave,GoalMouthY,Cross,HighLeft,SmallBoxLeft,OutOfBoxRight,OutOfBoxCentre,ThrowinSetPiece,KeeperSaveObox,PlayerCaughtOffside,FormationSlot,Volley,Longball,FirstTouch,GoalMouthZ,IntentionalGoalAssist,Red,BigChanceCreated,SetPiece,BlockedX,Length,VoidYellowCard,Obstruction,BoxLeft,ParriedDanger,StandingSave,PlayerPosition,DeepBoxRight,Foul,MissLeft,Head,ShotAssist,SixYardBlock,LeftFoot,Assisted,OneOnOne,MissHigh,BoxRight,RegularPlay,JerseyNumber,Yellow,MissRight,GoalDisallowed,KeeperSaveInSixYard,Offensive,Zone,BoxCentre,Throughball,OutfielderBlock,CaptainPlayerId,Blocked,TeamPlayerFormation,RightFoot,Hands,FromCorner,OutOfBoxDeepRight,Angle,FastBreak,OtherBodyPart,IndirectFreekickTaken,Feet,OutOfBoxDeepLeft,LeadingToAttempt,GoalKick,BigChance,InvolvedPlayers,LastMan,BlockedY,Defensive,HighCentre,Penalty,Collected,LowRight,DeepBoxLeft,Chipped,LowCentre,SmallBoxCentre,KeeperSaved,KeyPass,LowLeft,SmallBoxRight,RelatedEventId,ThrowIn,SavedOffline,IntentionalAssist,OwnGoal,CornerTaken,OverRun,LayOff,ThirtyFivePlusCentre,KeeperThrow,HighClaim,HighRight,KeeperMissed,KeeperSaveInTheBox,TeamFormation,DirectFreekick,AerialFoul,SecondYellow,FreekickTaken,LeadingToGoal
0,2.579628e+09,2,0,0.0,302,0.0,0.0,0,[],False,1,FirstHalf,32,Start,1,Successful,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
1,2.579628e+09,2,0,0.0,607,0.0,0.0,0,[],False,1,FirstHalf,32,Start,1,Successful,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
2,2.579628e+09,3,0,0.0,607,50.0,50.0,0,"[91, 117, 30, 35, 38, 215, 218]",True,1,FirstHalf,1,Pass,1,Successful,85693.0,32.5,44.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,Rémy Cabella,None,None,None,None,None,32.5,44.8,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,18.7,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,Back,None,None,None,None,None,None,None,None,None,None,3.33,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
3,2.579628e+09,4,0,2.0,607,36.5,49.7,0,"[91, 120, 124, 128, 36, 37, 217, 218]",True,1,FirstHalf,1,Pass,0,Unsuccessful,366875.0,71.7,88.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,Bafodé Diakité,None,None,None,None,None,71.7,88.0,None,None,None,None,None,

In [13]:
lille_events = events_all.loc[events_all.team_id==607].reset_index(drop=True)
lille_events

,id,event_id,minute,second,team_id,x,y,expanded_minute,satisfied_events_types,is_touch,period_value,period_display_name,type_value,type_display_name,outcome_type_value,outcome_type_display_name,player_id,end_x,end_y,blocked_x,blocked_y,goal_mouth_z,goal_mouth_y,is_shot,related_event_id,related_player_id,card_type_value,cardType.displayName,is_goal,match_id,is_own_goal,player_name,BlockedCross,HeadPass,ParriedSafe,OppositeRelatedEvent,IndividualPlay,PassEndX,PassEndY,DivingSave,GoalMouthY,Cross,HighLeft,SmallBoxLeft,OutOfBoxRight,OutOfBoxCentre,ThrowinSetPiece,KeeperSaveObox,PlayerCaughtOffside,FormationSlot,Volley,Longball,FirstTouch,GoalMouthZ,IntentionalGoalAssist,Red,BigChanceCreated,SetPiece,BlockedX,Length,VoidYellowCard,Obstruction,BoxLeft,ParriedDanger,StandingSave,PlayerPosition,DeepBoxRight,Foul,MissLeft,Head,ShotAssist,SixYardBlock,LeftFoot,Assisted,OneOnOne,MissHigh,BoxRight,RegularPlay,JerseyNumber,Yellow,MissRight,GoalDisallowed,KeeperSaveInSixYard,Offensive,Zone,BoxCentre,Throughball,OutfielderBlock,CaptainPlayerId,Blocked,TeamPlayerFormation,RightFoot,Hands,FromCorner,OutOfBoxDeepRight,Angle,FastBreak,OtherBodyPart,IndirectFreekickTaken,Feet,OutOfBoxDeepLeft,LeadingToAttempt,GoalKick,BigChance,InvolvedPlayers,LastMan,BlockedY,Defensive,HighCentre,Penalty,Collected,LowRight,DeepBoxLeft,Chipped,LowCentre,SmallBoxCentre,KeeperSaved,KeyPass,LowLeft,SmallBoxRight,RelatedEventId,ThrowIn,SavedOffline,IntentionalAssist,OwnGoal,CornerTaken,OverRun,LayOff,ThirtyFivePlusCentre,KeeperThrow,HighClaim,HighRight,KeeperMissed,KeeperSaveInTheBox,TeamFormation,DirectFreekick,AerialFoul,SecondYellow,FreekickTaken,LeadingToGoal
0,2.579628e+09,2,0,0.0,607,0.0,0.0,0,[],False,1,FirstHalf,32,Start,1,Successful,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
1,2.579628e+09,3,0,0.0,607,50.0,50.0,0,"[91, 117, 30, 35, 38, 215, 218]",True,1,FirstHalf,1,Pass,1,Successful,85693.0,32.5,44.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,Rémy Cabella,None,None,None,None,None,32.5,44.8,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,18.7,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,Back,None,None,None,None,None,None,None,None,None,None,3.33,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
2,2.579628e+09,4,0,2.0,607,36.5,49.7,0,"[91, 120, 124, 128, 36, 37, 217, 218]",True,1,FirstHalf,1,Pass,0,Unsuccessful,366875.0,71.7,88.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,Bafodé Diakité,None,None,None,None,None,71.7,88.0,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,45.2,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,Left,None,None,None,None,None,None,None,None,None,None,0.61,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
3,2.579628e+09,6,0,10.0,607,71.3,100.0,0,"[91, 212, 35, 38, 216]",True,1,FirstHalf,1,Pass,1,Successful,81662.0,55.0,92.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,Ismaily,None,None,None,None,None,55

In [14]:
pass_success = lille_events.loc[(lille_events.type_display_name=='Pass') & (lille_events.outcome_type_display_name=='Successful')]
for i in pass_success.index:
    lille_events.loc[i, ['pass_recipient_name']]= lille_events.iloc[i+1]['player_name']
    lille_events.loc[i, ['pass_recipient_id']] = lille_events.iloc[i+1]['player_id']
lille_events

,id,event_id,minute,second,team_id,x,y,expanded_minute,satisfied_events_types,is_touch,period_value,period_display_name,type_value,type_display_name,outcome_type_value,outcome_type_display_name,player_id,end_x,end_y,blocked_x,blocked_y,goal_mouth_z,goal_mouth_y,is_shot,related_event_id,related_player_id,card_type_value,cardType.displayName,is_goal,match_id,is_own_goal,player_name,BlockedCross,HeadPass,ParriedSafe,OppositeRelatedEvent,IndividualPlay,PassEndX,PassEndY,DivingSave,GoalMouthY,Cross,HighLeft,SmallBoxLeft,OutOfBoxRight,OutOfBoxCentre,ThrowinSetPiece,KeeperSaveObox,PlayerCaughtOffside,FormationSlot,Volley,Longball,FirstTouch,GoalMouthZ,IntentionalGoalAssist,Red,BigChanceCreated,SetPiece,BlockedX,Length,VoidYellowCard,Obstruction,BoxLeft,ParriedDanger,StandingSave,PlayerPosition,DeepBoxRight,Foul,MissLeft,Head,ShotAssist,SixYardBlock,LeftFoot,Assisted,OneOnOne,MissHigh,BoxRight,RegularPlay,JerseyNumber,Yellow,MissRight,GoalDisallowed,KeeperSaveInSixYard,Offensive,Zone,BoxCentre,Throughball,OutfielderBlock,CaptainPlayerId,Blocked,TeamPlayerFormation,RightFoot,Hands,FromCorner,OutOfBoxDeepRight,Angle,FastBreak,OtherBodyPart,IndirectFreekickTaken,Feet,OutOfBoxDeepLeft,LeadingToAttempt,GoalKick,BigChance,InvolvedPlayers,LastMan,BlockedY,Defensive,HighCentre,Penalty,Collected,LowRight,DeepBoxLeft,Chipped,LowCentre,SmallBoxCentre,KeeperSaved,KeyPass,LowLeft,SmallBoxRight,RelatedEventId,ThrowIn,SavedOffline,IntentionalAssist,OwnGoal,CornerTaken,OverRun,LayOff,ThirtyFivePlusCentre,KeeperThrow,HighClaim,HighRight,KeeperMissed,KeeperSaveInTheBox,TeamFormation,DirectFreekick,AerialFoul,SecondYellow,FreekickTaken,LeadingToGoal,pass_recipient_name,pass_recipient_id
0,2.579628e+09,2,0,0.0,607,0.0,0.0,0,[],False,1,FirstHalf,32,Start,1,Successful,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,NaN,NaN
1,2.579628e+09,3,0,0.0,607,50.0,50.0,0,"[91, 117, 30, 35, 38, 215, 218]",True,1,FirstHalf,1,Pass,1,Successful,85693.0,32.5,44.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,Rémy Cabella,None,None,None,None,None,32.5,44.8,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,18.7,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,Back,None,None,None,None,None,None,None,None,None,None,3.33,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,Bafodé Diakité,366875.0
2,2.579628e+09,4,0,2.0,607,36.5,49.7,0,"[91, 120, 124, 128, 36, 37, 217, 218]",True,1,FirstHalf,1,Pass,0,Unsuccessful,366875.0,71.7,88.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,Bafodé Diakité,None,None,None,None,None,71.7,88.0,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,45.2,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,Left,None,None,None,None,None,None,None,None,None,None,0.61,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,NaN,NaN
3,2.579628e+09,6,0,10.0,607,71.3,100.0,0,"[91, 212, 35, 38, 216]",True,1,FirstHalf,1,Pass,1,Successful,81662.0,55.0,92.3,NaN

In [15]:
lille_events.to_csv('Lille Events 2023-2024 Preprocessed.csv')